<a href="https://colab.research.google.com/github/sukruthhv/Google_Colab_Notebooks/blob/main/Yelp_Arizona_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Restaurant Recommendation System
## Theo Jeremiah

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [3]:
!pip install -U -q PyDrive

In [4]:
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 


In [5]:
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [6]:
link = "https://drive.google.com/file/d/1Yt1Cf3RIpd7GR8qdaxpLdWcqiRYsXA8s/view?usp=sharing"
id = link.split("/")[-2] 
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('yelp_review_Arizona.csv')
df = pd.read_csv('yelp_review_Arizona.csv')


In [7]:
#df = yelp[['review_id', 'user_id', 'business_id', 'text', 'stars', 'date']]

In [8]:
link = "https://drive.google.com/file/d/1qa5fibEn6KbGljUGc7ah451fi7XMdDAy/view?usp=sharing"
id = link.split("/")[-2] 
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('yelp_business.csv')
df_business = pd.read_csv('yelp_business.csv')

In [9]:
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26606 entries, 0 to 26605
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   review_id    26606 non-null  object 
 1   user_id      26606 non-null  object 
 2   business_id  26606 non-null  object 
 3   text         26606 non-null  object 
 4   stars        26606 non-null  float64
 5   date         26606 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.2+ MB


In [11]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   192609 non-null  object 
 1   name          192609 non-null  object 
 2   address       184927 non-null  object 
 3   city          192608 non-null  object 
 4   state         192609 non-null  object 
 5   postal_code   191950 non-null  object 
 6   latitude      192609 non-null  float64
 7   longitude     192609 non-null  float64
 8   stars         192609 non-null  float64
 9   review_count  192609 non-null  int64  
 10  is_open       192609 non-null  int64  
 11  attributes    163773 non-null  object 
 12  categories    192127 non-null  object 
 13  hours         147779 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 20.6+ MB


In [12]:
df_business.describe()

,latitude,longitude,stars,review_count,is_open
count,192609.000000,192609.000000,192609.000000,192609.000000,192609.000000
mean,38.541803,-97.594785,3.585627,33.538962,0.823040
std,4.941964,16.697725,1.018458,110.135224,0.381635
min,33.204642,-115.493471,1.000000,3.000000,0.000000
25%,33.637408,-112.274677,3.000000,4.000000,1.000000
50%,36.144815,-111.759324,3.500000,9.000000,1.000000
75%,43.602989,-79.983614,4.500000,25.000000,1.000000
max,51.299943,-72.911982,5.000000,8348.000000,1.000000


In [13]:
df_business.head()
#df_business.loc[df_business['business_id']=='2c9Vptks_vowLgVUMnCgjw']['name']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",NaN
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,NaN,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [14]:
df_business.nunique()

business_id     192609
name            145046
address         151976
city              1203
state               36
postal_code      17540
latitude        155162
longitude       150404
stars                9
review_count      1184
is_open              2
attributes       93976
categories       93385
hours            51566
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26606 entries, 0 to 26605
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   review_id    26606 non-null  object 
 1   user_id      26606 non-null  object 
 2   business_id  26606 non-null  object 
 3   text         26606 non-null  object 
 4   stars        26606 non-null  float64
 5   date         26606 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.2+ MB


In [16]:
#Check Null values in Dataframe
df.isnull().sum()

review_id      0
user_id        0
business_id    0
text           0
stars          0
date           0
dtype: int64

In [17]:
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [18]:
df.shape

(26606, 6)

### Select only stars and text

In [19]:
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [20]:
#Select only stars and text
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]

In [21]:
type(yelp_data)

pandas.core.frame.DataFrame

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.corpus import stopwords

for word in stopwords.words('english'):
    print(word)

import string
print(string.punctuation)

i
me
my
myself
we
our
ours
ourselves
you
you're
you've
you'll
you'd
your
yours
yourself
yourselves
he
him
his
himself
she
she's
her
hers
herself
it
it's
its
itself
they
them
their
theirs
themselves
what
which
who
whom
this
that
that'll
these
those
am
is
are
was
were
be
been
being
have
has
had
having
do
does
did
doing
a
an
the
and
but
if
or
because
as
until
while
of
at
by
for
with
about
against
between
into
through
during
before
after
above
below
to
from
up
down
in
out
on
off
over
under
again
further
then
once
here
there
when
where
why
how
all
any
both
each
few
more
most
other
some
such
no
nor
not
only
own
same
so
than
too
very
s
t
can
will
just
don
don't
should
should've
now
d
ll
m
o
re
ve
y
ain
aren
aren't
couldn
couldn't
didn
didn't
doesn
doesn't
hadn
hadn't
hasn
hasn't
haven
haven't
isn
isn't
ma
mightn
mightn't
mustn
mustn't
needn
needn't
shan
shan't
shouldn
shouldn't
wasn
wasn't
weren
weren't
won
won't
wouldn
wouldn't
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [24]:
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [25]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'youre',
 'youve',
 'youll',
 'youd',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'shes',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'thatll',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'fe

In [26]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

### Clean Text

In [27]:
df.iloc[0]['text']

"Came here while in town for a country concert.  What better way to start our day.  Everything was great.  The service was amazing from the time we walked in the door.  We were sat at a table and to our surprise there is a beer and whiskey tap at the table.  You pay by the ounce and there is a screen to show you what you've drank.  The food we had was great.  The whiskey burger was my favorite.  For you country fans this is a must when in Scottsdale!"

In [28]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",NaN
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,NaN,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [29]:
yelp_data

,business_id,user_id,stars,text
0,2c9Vptks_vowLgVUMnCgjw,ZwVz20be-hOZnyAbevyMyQ,4.0,Came here while in town for a country concert....
1,6nKR80xEGHYf2UxAe_Cu_g,91TB-gzcNyxFh46TL0pmnQ,5.0,Best barbecue this side of the Mississippi!!!!...
2,fbQaKW0Lte0JQ_opbnjdKg,Y_QBiZpATJoz8hKUfYF66A,5.0,Absolutely amazing. Think Chipotle for enchila...
3,r8764MtYyt8JhxMvrfM_xQ,_Jg-IA0M-GSjBlGu-wmejg,2.0,I was really disappointed with my most recent ...
4,QS3QxI7u5PRdtbGgI0-UsA,1CqkFliipv_X15WYn5aPfg,4.0,I grade sushi restaurants on 3 factors:\n- Qua...
...,...,...,...,...
26601,8nNB_yG1gzVpbqNH2EBucg,iDlkZO2iILS8Jwfdy7DP9A,3.0,Met a friend here for lunch. It was not as pa...
26602,R-sEoZXf37pGzLnid7s6wA,5monCcK5b_sRvUpOk0H1Sw,5.0,Love this place! One of the best local Mexican...
26603,SurnOSM2bVVN4-Js3G23RQ,llXWc7py2mDQwZrNrogSDg,4.0,I like the unique atmosphere of this place. bu...
26604,ms7cmWkPiSBDTnz-1sv2kQ,mRaaWyAFSwYGOT40T1-T9Q,2.0,"Oh Ticoz, I have been through a lot with you. ..."


In [30]:
yelp_data['text'] = yelp_data['text'].apply(text_process)

In [31]:
yelp_data['text']

0        Came town country concert better way start day...
1        Best barbecue side Mississippi Come car washed...
2        Absolutely amazing Think Chipotle enchiladas N...
3        really disappointed recent visit Vintage 95 ma...
4        grade sushi restaurants 3 factors Quality Exce...
                               ...                        
26601    Met friend lunch packed thought would nice wal...
26602    Love place One best local Mexican joints Ive s...
26603    like unique atmosphere place like space inside...
26604    Oh Ticoz lot good bad good last night worth ch...
26605     think owner owner Amys Baking Company soul mates
Name: text, Length: 26606, dtype: object

In [32]:
#Split train test for testing the model later
vld_size=0.15
X_train, X_valid, y_train, y_valid = train_test_split(yelp_data['text'], df['business_id'], test_size = vld_size) 

In [33]:
X_train

23677    Uncle bears pretty standard far breweries go f...
10979    Ive heard place whiledecided check outvery goo...
22891    Amazing fall bone pull apart fork melt mouth B...
24144    Ok heres update give little back info owner JJ...
17340    really loved place know reviews bad great expe...
                               ...                        
13623    Food location greatly expanded Although still ...
8721     friends invited us brunch become instant fans ...
10368    work close Switch come often employees town go...
8311     place good Think going neighbors BBQ place gou...
11543    gonna lie place pretty good gotten humanitaria...
Name: text, Length: 22615, dtype: object

In [34]:
X_valid

13493    Went Happy Hour sushi disappointing rolls sadl...
12295    say fan first sight Patio presence Immediately...
18556    Every salad menu AMAZING thin crust pizza die ...
9178     food restaurant consistently excellent went an...
5026     Really traditional bbq high quality tasty ribs...
                               ...                        
8686     rather eat place seriously tasty chicken fried...
4431     Good stuff fast freshcheckerboard roll favorit...
405      Trivia blast host Monday Tuesday Wednesday nig...
23884    coming toronto abundance amazing authentic tha...
21021    Karsens grill great venue Old Town escape tour...
Name: text, Length: 3991, dtype: object

In [35]:
y_train

23677    82I9SSilxh3zL8F1UQ45Xw
10979    cwWQB8LyFbxxiOzgfZdWCg
22891    VyVIneSU7XAWgMBllI6LnQ
24144    aVTcYZ1h7QoF_lDdH_RLqg
17340    oxv2UyI5yF6yawPU20tOxA
                  ...          
13623    wl0QZqAzr1DelslQ02JGCQ
8721     -Ht7HiGBox8lS1Y8IPjO8g
10368    -Ht7HiGBox8lS1Y8IPjO8g
8311     U6P56MjLrraN0JBNvOjxaA
11543    Noi53T0PWNEN9mQRS3-Ncg
Name: business_id, Length: 22615, dtype: object

In [36]:
y_valid

13493    fT6Uwl6abRQfiNgxre_qbg
12295    vjaHGc25CcyNPZTbjDa7Ww
18556    4LB4xtFAwfvJRachr6Eaxw
9178     UE3AvVnaPEBPneVahYT2eA
5026     2skQeu3C36VCiB653MIfrw
                  ...          
8686     wJiC71DNRiUBQAede-qVEw
4431     yqYtY3-Po4OVPafA9Z-Xyw
405      VbqtbvvtNOe-YKA61-fSNA
23884    JQYkVnUzS5vgFxZgCuW7ag
21021    bS9FdoDvInQe9J48wNFCcA
Name: business_id, Length: 3991, dtype: object

### Create two tables of user,text and business,text

In [37]:
yelp_data

,business_id,user_id,stars,text
0,2c9Vptks_vowLgVUMnCgjw,ZwVz20be-hOZnyAbevyMyQ,4.0,Came town country concert better way start day...
1,6nKR80xEGHYf2UxAe_Cu_g,91TB-gzcNyxFh46TL0pmnQ,5.0,Best barbecue side Mississippi Come car washed...
2,fbQaKW0Lte0JQ_opbnjdKg,Y_QBiZpATJoz8hKUfYF66A,5.0,Absolutely amazing Think Chipotle enchiladas N...
3,r8764MtYyt8JhxMvrfM_xQ,_Jg-IA0M-GSjBlGu-wmejg,2.0,really disappointed recent visit Vintage 95 ma...
4,QS3QxI7u5PRdtbGgI0-UsA,1CqkFliipv_X15WYn5aPfg,4.0,grade sushi restaurants 3 factors Quality Exce...
...,...,...,...,...
26601,8nNB_yG1gzVpbqNH2EBucg,iDlkZO2iILS8Jwfdy7DP9A,3.0,Met friend lunch packed thought would nice wal...
26602,R-sEoZXf37pGzLnid7s6wA,5monCcK5b_sRvUpOk0H1Sw,5.0,Love place One best local Mexican joints Ive s...
26603,SurnOSM2bVVN4-Js3G23RQ,llXWc7py2mDQwZrNrogSDg,4.0,like unique atmosphere place like space inside...
26604,ms7cmWkPiSBDTnz-1sv2kQ,mRaaWyAFSwYGOT40T1-T9Q,2.0,Oh Ticoz lot good bad good last night worth ch...


In [38]:
yelp_data.loc[yelp_data['user_id']=='ZwVz20be-hOZnyAbevyMyQ']

,business_id,user_id,stars,text
0,2c9Vptks_vowLgVUMnCgjw,ZwVz20be-hOZnyAbevyMyQ,4.0,Came town country concert better way start day...
1098,1qwkbN3wu3AGDxMTXVz2gA,ZwVz20be-hOZnyAbevyMyQ,5.0,Amazing Im golfer ended loving went family lov...
3909,6Iu4YDjAPrDRRrPCt_E2zA,ZwVz20be-hOZnyAbevyMyQ,1.0,saw many great reviews place decided try high ...
11491,_XN-GwzZwAyIqLKJsl2htg,ZwVz20be-hOZnyAbevyMyQ,4.0,Amazing Mexican Asian infusion good service sa...
19647,LAiBt7LobYM_0_EQrSvK4A,ZwVz20be-hOZnyAbevyMyQ,4.0,beautiful hotel excellent service Nick front a...


In [39]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [40]:
userid_df.head()

,user_id,text
0,ZwVz20be-hOZnyAbevyMyQ,Came town country concert better way start day...
1,91TB-gzcNyxFh46TL0pmnQ,Best barbecue side Mississippi Come car washed...
2,Y_QBiZpATJoz8hKUfYF66A,Absolutely amazing Think Chipotle enchiladas N...
3,_Jg-IA0M-GSjBlGu-wmejg,really disappointed recent visit Vintage 95 ma...
4,1CqkFliipv_X15WYn5aPfg,grade sushi restaurants 3 factors Quality Exce...


In [41]:
userid_df[userid_df['user_id']=='ZwVz20be-hOZnyAbevyMyQ']['text']

0        Came town country concert better way start day...
1098     Amazing Im golfer ended loving went family lov...
3909     saw many great reviews place decided try high ...
11491    Amazing Mexican Asian infusion good service sa...
19647    beautiful hotel excellent service Nick front a...
Name: text, dtype: object

In [42]:
business_df.head()

,business_id,text
0,2c9Vptks_vowLgVUMnCgjw,Came town country concert better way start day...
1,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue side Mississippi Come car washed...
2,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing Think Chipotle enchiladas N...
3,r8764MtYyt8JhxMvrfM_xQ,really disappointed recent visit Vintage 95 ma...
4,QS3QxI7u5PRdtbGgI0-UsA,grade sushi restaurants 3 factors Quality Exce...


### Join the text with for each user_id and business_id

In [43]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [44]:
business_df.head()

,text
business_id,
-050d_XIor1NpCuWkbIVaQ,Matts Big Breakfast Yes busy Yes tiny yes ther...
-1UMR00eXtwaeh59pEiDjA,go 3 stars instead 2 airport sat greeted unhap...
-4TMQnQJW1yd6NqGRDvAeA,disclaimer Ive privilege meeting many people w...
-6h3K1hj0d4DRcZNUtHDuw,went lunch today friend marlin special sounded...
-8QlV3b_9H4BAh6LgMIr1g,beloved Royal Taj returned Nandini ownership s...


In [45]:
userid_df.head()

,text
user_id,
--2HUmLkcNHZp0xw6AMBPg,place JAM Surfer vibe great eats love machaca ...
--4rAAfZnEIAKJE80aIiYg,pulled pork spicy bbq sauce impressed Probably...
--Nnm_506G_p8MxAOQna5w,Cant say burger anything special Taste ok Shak...
--ty7Z9fEt08E3dS3_qoSA,know think important trust Yelp kind reviews t...
-0IiMAZI2SsQ7VmyzJjokQ,Ever fan roadside attractions Americana Id rea...


In [46]:
userid_df.loc['ZwVz20be-hOZnyAbevyMyQ']['text']

'Came town country concert better way start day Everything great service amazing time walked door sat table surprise beer whiskey tap table pay ounce screen show drank food great whiskey burger favorite country fans must Scottsdale Amazing Im golfer ended loving went family loves golf planned staying hour since golf 6 hours later left far drive place right food amazing expect good brisket tacos chorizo sliders favorite Good quality food many unique items highly recommend place people ages teens loved bays nice roomy 7 us bay 10 people must try back soon saw many great reviews place decided try high hopes loved atmosphere kids loved Texas trivia game thats good thing could say steak came cooked way asked took back fix steak flavor covered butter thought steak even kids eat theres waitress asked daughter something wrong since ate bites explained like said ok walked away shrimp good pricey bugers ok end pushing us take home since ate almost none politely declined still came boxes trying g

In [47]:
#for word in userid_df[0]['text']

SyntaxError: ignored

## User Tfidf Vectorizer with 5000 Features (represent 88% of all words)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(10937, 5000)

In [50]:
userid_vectors

<10937x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1083887 stored elements in Compressed Sparse Row format>

In [51]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(1411, 5000)

# Matrix Factorization

In [52]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(10937, 1411)

In [53]:
userid_rating_matrix.head()

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,-F5mm0-YeCI7viSiOwVAAw,-FLnsWAa4AGEW4NgE8Fqew,-Ht7HiGBox8lS1Y8IPjO8g,-ITj6Pu8Gdw8MmLf0XBEKQ,-JwSuDqyFILFJmVpjrVbwA,-KIdCJnkt5N8rnnmWR5MQg,-Ma5JqzuI_3xIAtpNJfQ2g,-MuatiMmslPOvk9kOMyjkA,-PUPEpMsjBjk_rX5lFaWng,-RYIKWBSTaJ21B2UMyzI-Q,-ZBfr1BHvArFp1d6XH8jOQ,-bMZCfTK7fxFaURynKpBMA,-bd4BQcl1ekgo7avaFngIw,-cBQKodqi77Q0vk-9iDvQA,-hq09PjRpGcx4XXhrqh5xw,-kIscN8I29eXMPkvyyxmRQ,-o082vExIs0VVNSuZmiTQA,-oSII3bw90cvyLmgsHgmpg,-rCIoxp7pqN3xY7E3-v56Q,-raUM9T-1dKPiB5_vRmFYA,-rhH9sL3XGFpoJXcxUpEWA,-s5zZEv5u9XWhx1i5E0_pA,-s_U95xEEmJu5GK-srqpUA,-vCLrTTgw6pBufdarW8ynA,-yQHIYKXH3HAdhh1W52OMQ,01xTdrNUuTOAyH7NaRWcUA,024QUHVU7nlyW5y2BeeGLg,02EYqwh47uVxOaOKrnx8SQ,042IHd5KjHiMuBtGtugO_g,05BpxQYVbywxliQ9yA0CKw,...,y2e1DgIUBlERep7hrKz2-w,yA6dKNm_zl1ucZCnwW8ZCg,yAXWKWRTs4wNy-zrJgoeXQ,yCK1Ok69D7TSWJESZQZHiQ,yF9G4dJAm94pfnEWzPtPYQ,yFUvKpW66cNsYXD6inDsog,yLMSxHjK56Az-KtMQyG7LQ,yNPh5SO-7wr8HPpVCDPbXQ,yRvmDKX-Ocgd8zq_iMgujA,yUkOP3Tmdp1pWA7p3zHboQ,yX0pOWG_Be9TjGnm0aihVA,yXj1P-pc6CGDFSta-45L5Q,yZAGhlsrmupLbrgdKm6reg,ydqbNz2sHENV9XOElwyAYA,ydt5yifyYhF5s430llv_Hw,yet0Pww_ouW49T7V41ecSw,ylxqmxh2gO1yCpQkIk6o3A,ymtJtXwsJUClLNDepV1piQ,yqYtY3-Po4OVPafA9Z-Xyw,ytAsW7DYezxMMzqoO_-nIQ,yvwK4MO2qnSZE5ggYGyk1A,z4K95Y4P5nMrtJiDtXWDYA,z5KWNSDvgv-4I62P8l8Ynw,z6-reuC5BYf_Rth9gMBfgQ,zDyv4CoaYvAgQhgr1IK1sg,zJGtD3y-pAIGNId4codEEg,zNGYsFeG_7XEBodbtfux5Q,zOL260xaXDYZZPBItzALxQ,zS3-CKZVVvUYvasJnAg-aA,zUgDrRtGvK5ZTFlHCsTHwA,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--4rAAfZnEIAKJE80aIiYg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ty7Z9fEt08E3dS3_qoSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0IiMAZI2SsQ7VmyzJjokQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
userid_rating_matrix.shape

(10937, 1411)

In [55]:
type(userid_rating_matrix)

pandas.core.frame.DataFrame

In [56]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [57]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,110,1100,11am,12,13,14,15,150,1520,16,16th,17,18,19,1st,2,20,200,2030,21,22,23,24,25,250,26,27,2nd,2pm,3,...,yall,yard,yasu,yay,yea,yeah,year,years,yell,yelling,yellow,yellowtail,yelp,yelper,yelpers,yep,yes,yesterday,yet,yikes,yo,yogurt,york,yorker,young,younger,yuck,yum,yummy,yup,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.0,0.008223,0.007567,0.018716,0.0,0.0,0.0,0.01094,0.0,0.0,0.0,0.011105,0.0,0.0,0.0,0.0,0.009198,0.0,0.009935,0.0,0.0,0.0,0.0,0.0,0.015637,0.009498,0.008787,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.029225,0.0,0.0,0.0,0.0,0.006474,...,0.010862,0.0,0.0,0.0,0.0,0.018298,0.004872,0.003861,0.0,0.0,0.0,0.0,0.008812,0.0,0.007223,0.0,0.025324,0.0,0.01249,0.0,0.0,0.0,0.007761,0.0,0.0,0.0,0.0,0.009828,0.023916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1UMR00eXtwaeh59pEiDjA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.038729,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.151697,0.000000,0.0,0.0,0.0,0.0,0.039596,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-4TMQnQJW1yd6NqGRDvAeA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.031282,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.027100,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.025594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-6h3K1hj0d4DRcZNUtHDuw,0.0,0.028893,0.026588,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.016161,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.055626,0.015438,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.056872,...,0.000000,0.0,0.0,0.0,0.0,0.021432,0.000000,0.040700,0.0,0.0,0.0,0.0,0.030965,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.017266,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8QlV3b_9H4BAh6LgMIr1g,0.0,0.000000,0.017147,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.01887,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Gradient Decent Optimization

In [58]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,110,1100,11am,12,13,14,15,150,1520,16,16th,17,18,19,1st,2,20,200,2030,21,22,23,24,25,250,26,27,2nd,2pm,3,...,yall,yard,yasu,yay,yea,yeah,year,years,yell,yelling,yellow,yellowtail,yelp,yelper,yelpers,yep,yes,yesterday,yet,yikes,yo,yogurt,york,yorker,young,younger,yuck,yum,yummy,yup,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.0,0.008223,0.007567,0.018716,0.0,0.0,0.0,0.01094,0.0,0.0,0.0,0.011105,0.0,0.0,0.0,0.0,0.009198,0.0,0.009935,0.0,0.0,0.0,0.0,0.0,0.015637,0.009498,0.008787,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.029225,0.0,0.0,0.0,0.0,0.006474,...,0.010862,0.0,0.0,0.0,0.0,0.018298,0.004872,0.003861,0.0,0.0,0.0,0.0,0.008812,0.0,0.007223,0.0,0.025324,0.0,0.01249,0.0,0.0,0.0,0.007761,0.0,0.0,0.0,0.0,0.009828,0.023916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1UMR00eXtwaeh59pEiDjA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.038729,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.151697,0.000000,0.0,0.0,0.0,0.0,0.039596,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-4TMQnQJW1yd6NqGRDvAeA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.031282,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.027100,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.025594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-6h3K1hj0d4DRcZNUtHDuw,0.0,0.028893,0.026588,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.016161,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.055626,0.015438,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.056872,...,0.000000,0.0,0.0,0.0,0.0,0.021432,0.000000,0.040700,0.0,0.0,0.0,0.0,0.030965,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.017266,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8QlV3b_9H4BAh6LgMIr1g,0.0,0.000000,0.017147,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.01887,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0: # start doing this, only for those entries, where the ratings are filled by users. Sparse Matrix!!!
                    #Caculate the error function/cost function
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    #Updating the values of P & Q, such that the eij or cost function and hence error is reducing!!
                    #Basically we are calculating derivative of eij(error) w.r.t P first and then Q in the below 2 equations
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        ## Now that we have figured out the best set of paramters/Factors that can lead to minimum error using SGD, 
        ## we can now calc the right error/RMSE using regularization parameters for P & Q
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [60]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: user 3h 13min 53s, sys: 5.4 s, total: 3h 13min 58s
Wall time: 3h 14min 11s


In [73]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,110,1100,11am,12,13,14,15,150,1520,16,16th,17,18,19,1st,2,20,200,2030,21,22,23,24,25,250,26,27,2nd,2pm,3,...,yall,yard,yasu,yay,yea,yeah,year,years,yell,yelling,yellow,yellowtail,yelp,yelper,yelpers,yep,yes,yesterday,yet,yikes,yo,yogurt,york,yorker,young,younger,yuck,yum,yummy,yup,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.001158,0.046774,0.043362,0.041756,0.006478,0.002138,0.010029,0.023638,0.000542,0.001409,0.000750,0.033710,0.000435,0.020155,0.001093,0.001206,0.033249,0.001074,0.027486,0.003051,0.017109,0.000492,0.001061,0.000407,0.046995,0.078744,0.034059,0.000749,0.000209,0.005676,0.000637,0.000800,0.000995,0.013523,0.058090,0.000293,0.000438,0.001036,0.000166,0.075108,...,0.024777,0.000699,0.000140,0.006370,0.010359,0.042611,0.025279,0.035230,0.000220,0.000566,0.004363,0.002199,0.035735,0.000564,0.011619,0.001304,0.087830,0.002010,0.056844,0.005791,0.000323,0.001960,0.012342,0.000417,0.007704,0.000813,0.000850,0.044001,0.098376,0.000384,0.000139,0.000476,0.000238,0.013895,0.000119,0.000197,0.001806,0.000067,0.000085,0.004687
-1UMR00eXtwaeh59pEiDjA,0.000126,0.001169,0.001863,0.000566,0.000278,0.000095,0.000099,0.000047,0.000095,0.000339,0.000114,0.000283,0.000022,0.000701,0.000158,0.000362,0.002237,0.000238,0.000340,0.000198,0.000019,0.000204,0.000177,0.000334,0.000170,0.047089,0.002468,0.000016,0.000009,0.000121,0.000020,0.000258,0.000179,0.169652,0.000141,0.000059,0.000014,0.000248,0.000008,0.045083,...,0.000106,0.000176,0.000004,0.000100,0.000016,0.000490,0.000769,0.004019,0.000022,0.000017,0.000529,0.000043,0.001167,0.000020,0.000241,0.000010,0.003548,0.000284,0.004084,0.000029,0.000004,0.000161,0.000353,0.000022,0.000705,0.002011,0.000081,0.000669,0.000921,0.000024,0.000003,0.000444,0.000181,0.000015,0.000010,0.000197,0.000387,0.000018,0.000008,0.000711
-4TMQnQJW1yd6NqGRDvAeA,0.000497,0.004620,0.003707,0.001494,0.000228,0.000101,0.000558,0.000692,0.000350,0.001170,0.000325,0.000743,0.000147,0.004869,0.004051,0.000878,0.005298,0.000445,0.000107,0.000519,0.000582,0.000377,0.000426,0.000056,0.000760,0.009670,0.005698,0.000850,0.000342,0.000626,0.000154,0.000461,0.000697,0.001180,0.000411,0.000085,0.000135,0.000734,0.000374,0.014002,...,0.000129,0.001398,0.000058,0.000200,0.000282,0.001330,0.055112,0.007509,0.004763,0.000413,0.001454,0.000751,0.004840,0.000235,0.000804,0.012710,0.050755,0.001186,0.010724,0.000217,0.000570,0.000369,0.000381,0.000201,0.003782,0.000542,0.000109,0.002568,0.045691,0.000140,0.000332,0.000700,0.000394,0.000153,0.000077,0.004837,0.000070,0.000053,0.001040,0.000478
-6h3K1hj0d4DRcZNUtHDuw,0.000447,0.050076,0.039643,0.001207,0.000276,0.001007,0.000561,0.000198,0.000451,0.000763,0.000184,0.000182,0.000459,0.003296,0.005615,0.001339,0.021692,0.000535,0.000293,0.000455,0.000455,0.000386,0.000791,0.000259,0.000811,0.081302,0.024390,0.000477,0.000151,0.000620,0.000305,0.007039,0.000729,0.001423,0.000456,0.000186,0.000066,0.000764,0.000146,0.074456,...,0.000188,0.000219,0.018085,0.000568,0.000355,0.043662,0.002309,0.071001,0.000354,0.000293,0.000812,0.001209,0.053601,0.000259,0.002993,0.000742,0.003843,0.007069,0.017197,0.000138,0.000221,0.002954,0.000515,0.000103,0.003216,0.000484,0.000670,0.027621,0.011289,0.000384,0.002316,0.002917,0.000074,0.000897,0.000926,0.000124,0.000099,0.000040,0.007819,0.001039
-8QlV3b_9H4BAh6LgMIr1g,0.000031,0.027002,0.043027,0.001043,0.000206,0.000046,0.000047,0.000084,0.000195,0.018048,0.000204,0.000020,0.000508,0.001096,0.000322,0.000293,0.005885,0.000107,0.000124,0.000168,0.000047,0.000032,0.002612,0.000023,0.000512,0.005953,0.010976,0.000163,0.000138,0.005799,0.000168,0.002407,0.000321,0.000604,0.000099,0.000040,0.000052,0.006688,0.000552,0.015662,...,0.000531,0.000498,0.000020,0.000136,0.000068,0.000618,0.031493,0.004186,0.000056,0.000203,0.000259,0.000063,0.011812,0.000172,0.000698,0.000111,0.00165

In [74]:
Q.iloc[0].sort_values(ascending=False).head(10)

matts        1.077482
breakfast    0.659543
wait         0.529304
food         0.428016
place        0.405557
eggs         0.362205
good         0.354656
bacon        0.312531
like         0.274301
big          0.265078
Name: -050d_XIor1NpCuWkbIVaQ, dtype: float64

#### SHV implementation using SVD

In [75]:
userid_rating_matrix

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,-F5mm0-YeCI7viSiOwVAAw,-FLnsWAa4AGEW4NgE8Fqew,-Ht7HiGBox8lS1Y8IPjO8g,-ITj6Pu8Gdw8MmLf0XBEKQ,-JwSuDqyFILFJmVpjrVbwA,-KIdCJnkt5N8rnnmWR5MQg,-Ma5JqzuI_3xIAtpNJfQ2g,-MuatiMmslPOvk9kOMyjkA,-PUPEpMsjBjk_rX5lFaWng,-RYIKWBSTaJ21B2UMyzI-Q,-ZBfr1BHvArFp1d6XH8jOQ,-bMZCfTK7fxFaURynKpBMA,-bd4BQcl1ekgo7avaFngIw,-cBQKodqi77Q0vk-9iDvQA,-hq09PjRpGcx4XXhrqh5xw,-kIscN8I29eXMPkvyyxmRQ,-o082vExIs0VVNSuZmiTQA,-oSII3bw90cvyLmgsHgmpg,-rCIoxp7pqN3xY7E3-v56Q,-raUM9T-1dKPiB5_vRmFYA,-rhH9sL3XGFpoJXcxUpEWA,-s5zZEv5u9XWhx1i5E0_pA,-s_U95xEEmJu5GK-srqpUA,-vCLrTTgw6pBufdarW8ynA,-yQHIYKXH3HAdhh1W52OMQ,01xTdrNUuTOAyH7NaRWcUA,024QUHVU7nlyW5y2BeeGLg,02EYqwh47uVxOaOKrnx8SQ,042IHd5KjHiMuBtGtugO_g,05BpxQYVbywxliQ9yA0CKw,...,y2e1DgIUBlERep7hrKz2-w,yA6dKNm_zl1ucZCnwW8ZCg,yAXWKWRTs4wNy-zrJgoeXQ,yCK1Ok69D7TSWJESZQZHiQ,yF9G4dJAm94pfnEWzPtPYQ,yFUvKpW66cNsYXD6inDsog,yLMSxHjK56Az-KtMQyG7LQ,yNPh5SO-7wr8HPpVCDPbXQ,yRvmDKX-Ocgd8zq_iMgujA,yUkOP3Tmdp1pWA7p3zHboQ,yX0pOWG_Be9TjGnm0aihVA,yXj1P-pc6CGDFSta-45L5Q,yZAGhlsrmupLbrgdKm6reg,ydqbNz2sHENV9XOElwyAYA,ydt5yifyYhF5s430llv_Hw,yet0Pww_ouW49T7V41ecSw,ylxqmxh2gO1yCpQkIk6o3A,ymtJtXwsJUClLNDepV1piQ,yqYtY3-Po4OVPafA9Z-Xyw,ytAsW7DYezxMMzqoO_-nIQ,yvwK4MO2qnSZE5ggYGyk1A,z4K95Y4P5nMrtJiDtXWDYA,z5KWNSDvgv-4I62P8l8Ynw,z6-reuC5BYf_Rth9gMBfgQ,zDyv4CoaYvAgQhgr1IK1sg,zJGtD3y-pAIGNId4codEEg,zNGYsFeG_7XEBodbtfux5Q,zOL260xaXDYZZPBItzALxQ,zS3-CKZVVvUYvasJnAg-aA,zUgDrRtGvK5ZTFlHCsTHwA,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4rAAfZnEIAKJE80aIiYg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--Nnm_506G_p8MxAOQna5w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ty7Z9fEt08E3dS3_qoSA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0IiMAZI2SsQ7VmyzJjokQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [76]:
userid_rating_matrix=userid_rating_matrix.fillna(0)
userid_rating_matrix#=userid_rating_matrix.values
#userid_rating_matrix_np_mean=np.mean(userid_rating_matrix_np,axis=1)
#userid_rating_matrix_np=userid_rating_matrix_np-userid_rating_matrix_np_mean.reshape(-1,1)
#userid_rating_matrix_np

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,-F5mm0-YeCI7viSiOwVAAw,-FLnsWAa4AGEW4NgE8Fqew,-Ht7HiGBox8lS1Y8IPjO8g,-ITj6Pu8Gdw8MmLf0XBEKQ,-JwSuDqyFILFJmVpjrVbwA,-KIdCJnkt5N8rnnmWR5MQg,-Ma5JqzuI_3xIAtpNJfQ2g,-MuatiMmslPOvk9kOMyjkA,-PUPEpMsjBjk_rX5lFaWng,-RYIKWBSTaJ21B2UMyzI-Q,-ZBfr1BHvArFp1d6XH8jOQ,-bMZCfTK7fxFaURynKpBMA,-bd4BQcl1ekgo7avaFngIw,-cBQKodqi77Q0vk-9iDvQA,-hq09PjRpGcx4XXhrqh5xw,-kIscN8I29eXMPkvyyxmRQ,-o082vExIs0VVNSuZmiTQA,-oSII3bw90cvyLmgsHgmpg,-rCIoxp7pqN3xY7E3-v56Q,-raUM9T-1dKPiB5_vRmFYA,-rhH9sL3XGFpoJXcxUpEWA,-s5zZEv5u9XWhx1i5E0_pA,-s_U95xEEmJu5GK-srqpUA,-vCLrTTgw6pBufdarW8ynA,-yQHIYKXH3HAdhh1W52OMQ,01xTdrNUuTOAyH7NaRWcUA,024QUHVU7nlyW5y2BeeGLg,02EYqwh47uVxOaOKrnx8SQ,042IHd5KjHiMuBtGtugO_g,05BpxQYVbywxliQ9yA0CKw,...,y2e1DgIUBlERep7hrKz2-w,yA6dKNm_zl1ucZCnwW8ZCg,yAXWKWRTs4wNy-zrJgoeXQ,yCK1Ok69D7TSWJESZQZHiQ,yF9G4dJAm94pfnEWzPtPYQ,yFUvKpW66cNsYXD6inDsog,yLMSxHjK56Az-KtMQyG7LQ,yNPh5SO-7wr8HPpVCDPbXQ,yRvmDKX-Ocgd8zq_iMgujA,yUkOP3Tmdp1pWA7p3zHboQ,yX0pOWG_Be9TjGnm0aihVA,yXj1P-pc6CGDFSta-45L5Q,yZAGhlsrmupLbrgdKm6reg,ydqbNz2sHENV9XOElwyAYA,ydt5yifyYhF5s430llv_Hw,yet0Pww_ouW49T7V41ecSw,ylxqmxh2gO1yCpQkIk6o3A,ymtJtXwsJUClLNDepV1piQ,yqYtY3-Po4OVPafA9Z-Xyw,ytAsW7DYezxMMzqoO_-nIQ,yvwK4MO2qnSZE5ggYGyk1A,z4K95Y4P5nMrtJiDtXWDYA,z5KWNSDvgv-4I62P8l8Ynw,z6-reuC5BYf_Rth9gMBfgQ,zDyv4CoaYvAgQhgr1IK1sg,zJGtD3y-pAIGNId4codEEg,zNGYsFeG_7XEBodbtfux5Q,zOL260xaXDYZZPBItzALxQ,zS3-CKZVVvUYvasJnAg-aA,zUgDrRtGvK5ZTFlHCsTHwA,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4rAAfZnEIAKJE80aIiYg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--Nnm_506G_p8MxAOQna5w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ty7Z9fEt08E3dS3_qoSA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0IiMAZI2SsQ7VmyzJjokQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [77]:
from scipy.sparse.linalg import svds
U, sigma,Vt = svds(userid_rating_matrix,k=50)

In [78]:
sigma = np.diag(sigma)
sigma

array([[32.26335418,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 32.35262227,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 32.42532641, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 44.55591142,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        47.30735082,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 54.63093673]])

In [79]:
all_predictions = np.dot(np.dot(U,sigma),Vt) #+ userid_rating_matrix_np.reshape(-1,1)
all_predictions_df = pd.DataFrame(all_predictions,columns=userid_rating_matrix.columns,index=userid_rating_matrix.index)
all_predictions_df

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,-F5mm0-YeCI7viSiOwVAAw,-FLnsWAa4AGEW4NgE8Fqew,-Ht7HiGBox8lS1Y8IPjO8g,-ITj6Pu8Gdw8MmLf0XBEKQ,-JwSuDqyFILFJmVpjrVbwA,-KIdCJnkt5N8rnnmWR5MQg,-Ma5JqzuI_3xIAtpNJfQ2g,-MuatiMmslPOvk9kOMyjkA,-PUPEpMsjBjk_rX5lFaWng,-RYIKWBSTaJ21B2UMyzI-Q,-ZBfr1BHvArFp1d6XH8jOQ,-bMZCfTK7fxFaURynKpBMA,-bd4BQcl1ekgo7avaFngIw,-cBQKodqi77Q0vk-9iDvQA,-hq09PjRpGcx4XXhrqh5xw,-kIscN8I29eXMPkvyyxmRQ,-o082vExIs0VVNSuZmiTQA,-oSII3bw90cvyLmgsHgmpg,-rCIoxp7pqN3xY7E3-v56Q,-raUM9T-1dKPiB5_vRmFYA,-rhH9sL3XGFpoJXcxUpEWA,-s5zZEv5u9XWhx1i5E0_pA,-s_U95xEEmJu5GK-srqpUA,-vCLrTTgw6pBufdarW8ynA,-yQHIYKXH3HAdhh1W52OMQ,01xTdrNUuTOAyH7NaRWcUA,024QUHVU7nlyW5y2BeeGLg,02EYqwh47uVxOaOKrnx8SQ,042IHd5KjHiMuBtGtugO_g,05BpxQYVbywxliQ9yA0CKw,...,y2e1DgIUBlERep7hrKz2-w,yA6dKNm_zl1ucZCnwW8ZCg,yAXWKWRTs4wNy-zrJgoeXQ,yCK1Ok69D7TSWJESZQZHiQ,yF9G4dJAm94pfnEWzPtPYQ,yFUvKpW66cNsYXD6inDsog,yLMSxHjK56Az-KtMQyG7LQ,yNPh5SO-7wr8HPpVCDPbXQ,yRvmDKX-Ocgd8zq_iMgujA,yUkOP3Tmdp1pWA7p3zHboQ,yX0pOWG_Be9TjGnm0aihVA,yXj1P-pc6CGDFSta-45L5Q,yZAGhlsrmupLbrgdKm6reg,ydqbNz2sHENV9XOElwyAYA,ydt5yifyYhF5s430llv_Hw,yet0Pww_ouW49T7V41ecSw,ylxqmxh2gO1yCpQkIk6o3A,ymtJtXwsJUClLNDepV1piQ,yqYtY3-Po4OVPafA9Z-Xyw,ytAsW7DYezxMMzqoO_-nIQ,yvwK4MO2qnSZE5ggYGyk1A,z4K95Y4P5nMrtJiDtXWDYA,z5KWNSDvgv-4I62P8l8Ynw,z6-reuC5BYf_Rth9gMBfgQ,zDyv4CoaYvAgQhgr1IK1sg,zJGtD3y-pAIGNId4codEEg,zNGYsFeG_7XEBodbtfux5Q,zOL260xaXDYZZPBItzALxQ,zS3-CKZVVvUYvasJnAg-aA,zUgDrRtGvK5ZTFlHCsTHwA,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,-0.128558,0.009017,0.033721,-0.052600,-0.001768,0.007659,-0.015792,0.094012,0.019127,-0.004400,0.000732,-0.021546,0.016372,-0.001886,-0.003409,0.007419,0.003594,0.006651,-0.008693,0.008959,-0.038634,0.005178,0.052363,0.001310,0.012878,0.060117,-0.024868,0.017524,-0.020733,0.005758,-0.117816,-0.020063,0.005104,-0.013374,-0.070428,0.004159,-0.022677,0.024682,-0.030801,0.001489,...,0.001928,-0.015290,0.000751,-0.014230,0.028452,-0.017533,-0.081336,0.168444,-0.001616,0.067345,0.018528,0.027369,0.014515,0.000912,-0.003551,-0.005943,-0.001382,0.057231,0.029006,-0.034178,0.002128,0.052425,0.036842,0.004052,0.006246,0.086122,0.012538,0.002436,-0.000218,-0.003058,0.005109,-0.004630,0.032924,-0.001026,0.028514,-0.010530,0.087755,-0.008754,0.025178,-0.034519
--4rAAfZnEIAKJE80aIiYg,-0.000587,0.001438,-0.002459,0.000194,-0.000560,0.003733,-0.010974,0.007665,-0.005812,0.002504,-0.000251,-0.014525,-0.011567,0.007824,0.000225,-0.001546,-0.002918,0.006577,0.002086,0.000728,0.013921,-0.002247,0.022301,-0.001354,0.008148,-0.020444,-0.002187,0.001559,0.001735,0.000981,0.063323,0.000774,0.000533,0.008509,0.053515,0.000639,-0.002107,0.006592,0.011683,-0.002105,...,0.000488,-0.001327,0.003821,0.004572,0.004846,0.005450,0.006156,-0.019011,-0.001552,0.000591,0.003856,0.001016,-0.014108,-0.003044,-0.001328,-0.001806,0.001433,-0.001568,0.006380,0.003038,-0.001081,0.002857,0.014807,0.129269,0.007620,-0.018760,0.002018,0.000674,-0.000188,0.000157,-0.002856,-0.000804,0.001949,-0.000690,-0.001775,0.000686,0.019346,0.017982,0.004729,0.005068
--Nnm_506G_p8MxAOQna5w,-0.015813,0.000178,0.007488,0.000109,0.001685,0.006088,0.016797,-0.009760,0.011100,0.002037,0.002995,-0.009164,0.034736,0.013381,0.000531,-0.000028,-0.001172,0.006891,0.000112,0.000486,0.014699,-0.000793,0.027723,-0.003345,0.002900,0.002938,0.009833,0.005963,-0.002055,-0.003515,-0.009109,-0.001953,0.001799,-0.001351,-0.064416,-0.001662,0.004077,0.000198,0.021351,0.004220,...,0.000618,-0.000332,-0.001719,0.003704,0.007479,0.000918,0.000262,0.142954,0.001613,0.002252,0.0

### Export Model

In [88]:
# Store P, Q and vectorizer in pickle file
import pickle
output = open('yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()
from google.colab import files
files.download('yelp_recommendation_model_8.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [91]:
output_1=open('Q_file.pkl','wb')
pickle.dump(Q,output_1)
output_1.close()
files.download('Q_file.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:
words = "i want to have dinner with beautiful views"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
len(userid_vectorizer.get_feature_names())

5000

## Prediction for input text

In [85]:
words = "Indian cuisine"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

print(test_v_df)
print(test_v_df.shape)
predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
print(Q.T.shape)
print("Q.T = ")
print(Q.T)
print("#########")
print(predictItemRating)
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]
print(topRecommendations)

for i in topRecommendations.index:
    print(df_business[df_business['business_id']==i]['name'].iloc[0])
    print(df_business[df_business['business_id']==i]['categories'].iloc[0])
    print(str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
    print('')


     0    1   10  100  1000  101  ...  zinc  zipps  ziti  zookz  ztejas  zucchini
0  0.0  0.0  0.0  0.0   0.0  0.0  ...   0.0    0.0   0.0    0.0     0.0       0.0

[1 rows x 5000 columns]
(1, 5000)
(5000, 1411)
Q.T = 
business_id  -050d_XIor1NpCuWkbIVaQ  ...  zuVvDYJkKAbXQTTBauAqJQ
0                          0.001158  ...                0.000137
1                          0.046774  ...                0.001904
10                         0.043362  ...                0.002239
100                        0.041756  ...                0.000951
1000                       0.006478  ...                0.013797
...                             ...  ...                     ...
zipps                      0.000197  ...                0.000061
ziti                       0.001806  ...                0.000021
zookz                      0.000067  ...                0.000065
ztejas                     0.000085  ...                0.005923
zucchini                   0.004687  ...                0.000958



In [71]:
df_business[df_business['business_id']=="dSnQTUJC0tCHadU1AGzvrw"]['name'].iloc[0]

"Isabella's Kitchen"

In [72]:
pd.Da

AttributeError: ignored